# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phạm Nguyễn Quốc Thanh
- MSSV: 21127428
- Lớp: 21CLC08

# Import

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown as md

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)
X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)
Salary_train = train.iloc[:, [-1]].to_numpy() 
Salary_test = test.iloc[:, [-1]]
first_11_data_train = train.iloc[:, :11].to_numpy()
first_11_data_test = test.iloc[:, :11].to_numpy()
first_3_data = train.iloc[:3, :3]
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel())) 

def k_fold_range(k_fold, n):
    range_list = []
    div, remain = n // k_fold, n % k_fold
    for i in range(k_fold):
        if i == k_fold - 1:
            range_list.append((i * div, (i + 1) * div + remain))
        else:
            range_list.append((i * div, (i + 1) * div))
    return range_list

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [197]:
# Phần code cho yêu cầu 1a
lr_1a = OLSLinearRegression().fit(first_11_data_train, y_train.to_numpy())
# required_1a = OLSLinearRegression().fit(first_11_data_test, y_test.to_numpy())
# required_1a.get_params()



(2248,)
(2248, 11)


In [118]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr_1a.predict(first_11_data_test)
print("MAE: ", mae(y_test.to_numpy().reshape(-1, 1), y_hat.reshape(-1, 1)))
np.set_printoptions(suppress=True, precision=3)
weights = lr_1a.get_params()
print("Weights:\n", weights.reshape(-1, 1))

MAE:  104863.77754032997
Weights:
 [[-22756.513]
 [   804.503]
 [  1294.655]
 [-91781.898]
 [ 23182.389]
 [  1437.549]
 [ -8570.662]
 [   147.858]
 [   152.888]
 [   117.222]
 [ 34552.286]]


In [100]:
first_11_columns = train.columns[:11].to_numpy()
results = zip(first_11_columns, weights)

display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = " + " + ".join(f"{w:.3f} x {f}" for f, w in results)))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = -22756.513 x Gender + 804.503 x 10percentage + 1294.655 x 12percentage + -91781.898 x CollegeTier + 23182.389 x Degree + 1437.549 x collegeGPA + -8570.662 x CollegeCityTier + 147.858 x English + 152.888 x Logical + 117.222 x Quant + 34552.286 x Domain

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = \text{Gender} \times -31888.616 + \text{10percentage} \times 1947.808 + \text{12percentage} \times 100.178 + \text{CollegeTier} \times -72462.209 + \text{Degree} \times -11070.036 + \text{collegeGPA} \times 851.434 + \text{CollegeCityTier} \times -6642.376 + \text{English} \times 70.451 + \text{Logical} \times -95.119 + \text{Quant} \times 379.677 + \text{Domain} \times 41249.229 $$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [204]:
c = X_train.iloc[:, 18]
lr_test = OLSLinearRegression().fit(c.to_numpy().reshape(-1, 1), y_train.to_numpy().reshape(-1, 1))
y_hat = lr_test.predict(X_test.iloc[:, 18].to_numpy().reshape(-1, 1))
print('MAE: ', mae(y_test.to_numpy().reshape(-1, 1), y_hat))

MAE:  299180.6403828239


In [222]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
X_train_1b_all = train.iloc[:, -6:].to_numpy()
np.random.shuffle(X_train_1b_all)

k_fold = 5
range_list = k_fold_range(k_fold, X_train_1b_all.shape[0])
avg_MAE = np.zeros((1, 5))
for i in range(5):
    idx = 18
    for j in range(5):
        for fold in range_list:
            lr_1b = OLSLinearRegression().fit(X_train_1b_all[fold[0]:fold[1], j].reshape(-1, 1), X_train_1b_all[fold[0]:fold[1], -1].reshape(-1, 1))
            y_hat = lr_1b.predict(X_train.iloc[:,idx].to_numpy().reshape(-1, 1))
            avg_MAE[0][j] += mae(y_train.to_numpy().reshape(-1, 1),y_hat)/5
        idx += 1
avg_MAE /= 5
personalities_columns = X_train.columns[-5:].to_numpy()
df = pd.DataFrame(list(zip(personalities_columns.flatten(), avg_MAE.flatten())), columns =['Mô hình với 1 đặc trưng', 'MAE'])
def format_float(x):
    return '{:.3f}'.format(x)
df['MAE'] = df['MAE'].apply(format_float)
print(df)
#Nueroticism là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng         MAE
0       conscientiousness  306179.646
1           agreeableness  300483.563
2            extraversion  306852.588
3             nueroticism  299514.381
4   openess_to_experience  303048.427


In [212]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
#Nueroticism columns
neuro = X_train.loc[:, 'nueroticism']
best_personality_feature_model = OLSLinearRegression().fit(neuro.to_numpy().reshape(-1, 1), y_train)

In [217]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
y_hat = best_personality_feature_model.predict(X_train.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1))
print("MAE: ", mae(y_train, y_hat))
print(best_personality_feature_model.w)

MAE:  299177.37436454324
[-56546.304]


In [218]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = neuroticism x -56546.304"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = neuroticism x -56546.304

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [234]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
X_train_1c = train.loc[:, ['English', 'Logical', 'Quant', 'Salary']].to_numpy()
np.random.shuffle(X_train_1c)

k_fold = 5
range_list = k_fold_range(n = X_train_1c.shape[0], k_fold = k_fold)
avg_MAE = np.zeros((1, 5))

for i in range(5):
    idx = 7
    for j in range(3):
        for fold in range_list:
            lr_1c = OLSLinearRegression().fit(X_train_1c[fold[0]:fold[1], j].reshape(-1, 1), X_train_1c[fold[0]:fold[1], -1].reshape(-1, 1))
            y_hat = lr_1c.predict(X_train.iloc[:, idx].to_numpy().reshape(-1, 1))
            avg_MAE[0][j] += mae(y_train.to_numpy().reshape(-1, 1), y_hat)/5
        idx += 1
avg_MAE /= 5 
elq_columns = X_train.columns[7:10].to_numpy()
df = pd.DataFrame(list(zip(elq_columns.flatten(), avg_MAE.flatten())), columns =['Mô hình với 1 đặc trưng', 'MAE'])
def format_float(x):
    return '{:.3f}'.format(x)
df['MAE'] = df['MAE'].apply(format_float)
print(df)
#Quant là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng         MAE
0                 English  122398.622
1                 Logical  120728.908
2                   Quant  118567.220


In [236]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
quant = X_train.loc[:, 'Quant']
best_personality_feature_model = OLSLinearRegression().fit(quant.to_numpy().reshape(-1, 1), y_train)

In [238]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
y_hat = best_personality_feature_model.predict(X_train.loc[:, 'Quant'].to_numpy().reshape(-1, 1))
print("MAE: ", mae(y_train, y_hat))
print(best_personality_feature_model.w)

MAE:  118049.00716605876
[585.895]


In [239]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = Quant x 585.895"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = Quant x 585.895

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [ ]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [ ]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [ ]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$